In [16]:
import pandas as pd
import numpy as np
import shapely
import matplotlib.pylab as plt
import geopandas as gpd
import fiona
import rtree

from geopandas import GeoDataFrame
from fiona.crs import from_epsg
from shapely.geometry import Point

%matplotlib inline

In [2]:
path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/Cleaned_DataSets/"
q_score = pd.read_csv(path + "streetscore_newyorkcity.csv")
q_score.head()

,latitude,longitude,q-score
0,40.700909,-74.013504,11.062166
1,40.701000,-74.013878,10.818611
2,40.701080,-74.012878,12.677955
3,40.701187,-74.013268,11.417325
4,40.701244,-74.012115,25.199091


In [3]:
crs = None
geometry = [Point(xy) for xy in zip(q_score.longitude, q_score.latitude)]
qscore_geo = GeoDataFrame(q_score, crs=crs, geometry=geometry)
qscore_geo.crs = fiona.crs.from_epsg(4326)

In [4]:
qscore_geo.head()

,latitude,longitude,q-score,geometry
0,40.700909,-74.013504,11.062166,POINT (-74.013504 40.700909)
1,40.701000,-74.013878,10.818611,POINT (-74.01387800000001 40.701)
2,40.701080,-74.012878,12.677955,POINT (-74.012878 40.70108)
3,40.701187,-74.013268,11.417325,POINT (-74.01326800000001 40.701187)
4,40.701244,-74.012115,25.199091,POINT (-74.01211500000001 40.701244)


In [5]:
shape_path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/bk_mappluto_16v2/BKMapPLUTO.shp"
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.to_crs(epsg=4326, inplace=True)

In [6]:
ct_shape.head()

,APPBBL,APPDate,Address,AreaSource,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BldgDepth,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.000000e+00,None,BEACH RESERVATION,4,2241.0,2241.0,3.070360e+09,0,V0,0.00,...,0,0,None,11224,R3-1,None,None,None,28b,"POLYGON ((-74.012372767644 40.57744691763495, ..."
1,0.000000e+00,None,BEACH 46 STREET,4,822.0,822.0,3.070370e+09,0,V0,0.00,...,0,0,None,11224,R3-1,None,None,None,28b,POLYGON ((-74.01237940630934 40.57657598861726...
2,0.000000e+00,None,JONES WALK,4,74250.0,74250.0,3.086960e+09,0,V1,0.00,...,0,0,None,11224,C7,None,None,None,28d,POLYGON ((-73.97880723690464 40.57467706266272...
3,0.000000e+00,None,3753 MERMAID AVENUE,2,8651.0,33253.0,3.070290e+09,3610,C0,32.00,...,0,1920,None,11224,R3-1,None,None,None,28b,POLYGON ((-74.00473188788037 40.57469753965819...
4,3.070490e+09,12/19/2000,3102 MERMAID AVENUE,2,8012.0,19652.0,3.070490e+09,1854,A5,40.37,...,0,1999,None,11224,R6,None,None,None,28d,POLYGON ((-73.99754223950133 40.57492465276304...


In [7]:
ct_shape.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AreaSource',
       u'AssessLand',  u'AssessTot',        u'BBL',   u'BldgArea',
        u'BldgClass',  u'BldgDepth',  u'BldgFront',      u'Block',
         u'BoroCode',    u'Borough',   u'BsmtCode',   u'BuiltFAR',
           u'CB2010',         u'CD',     u'CT2010',    u'ComArea',
          u'CommFAR',    u'CondoNo',    u'Council',  u'EDesigNum',
        u'Easements', u'ExemptLand',  u'ExemptTot',        u'Ext',
         u'FacilFAR', u'FactryArea',   u'FireComp', u'GarageArea',
       u'HealthArea',   u'HistDist', u'IrrLotCode',    u'LandUse',
         u'Landmark',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType',  u'LtdHeight', u'MAPPLUTO_F',
         u'NumBldgs',  u'NumFloors', u'OfficeArea',  u'OtherArea',
         u'Overlay1',   u'Overlay2',  u'OwnerName',  u'OwnerType',
       u'PLUTOMapID', u'PolicePrct',   u'ProxCode',    u'ResArea',
         u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Le

In [8]:
BK_Shape = ct_shape.drop([u'Borough', u'CD', u'CB2010', u'Council', u'SanitBoro', u'SanitDistr', u'SanitSub', 
                          u'Overlay1',u'Overlay2', u'SPDist1', u'SPDist2', u'SPDist3', u'LtdHeight', u'SplitZone', 
                          u'Easements', u'OwnerName', u'AreaSource', u'IrrLotCode', u'BsmtCode', u'AssessLand', 
                          u'AssessTot', u'ExemptLand', u'ExemptTot', u'YearAlter1', u'YearAlter2',u'BuiltFAR', 
                          u'ResidFAR', u'CommFAR', u'FacilFAR', u'BoroCode', u'BBL', u'CondoNo', u'Tract2010', 
                          u'ZoneMap', u'ZMCode', u'Sanborn', u'TaxMap', u'EDesigNum', u'APPBBL', u'APPDate', 
                          u'PLUTOMapID', u'Version', u'Ext', u'HistDist', u'Landmark', u'ZoneDist2', u'ZoneDist3', 
                          u'ZoneDist4'], axis = 1)

In [9]:
BK_Shape.columns

Index([   u'Address',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',     u'CT2010',    u'ComArea',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
          u'LandUse',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',  u'OwnerType',
       u'PolicePrct',   u'ProxCode',    u'ResArea', u'RetailArea',
       u'SHAPE_Area', u'SHAPE_Leng', u'SchoolDist',  u'StrgeArea',
         u'UnitsRes', u'UnitsTotal',     u'XCoord',     u'YCoord',
        u'YearBuilt',    u'ZipCode',  u'ZoneDist1',   u'geometry'],
      dtype='object')

In [10]:
BK_Shape_2 = BK_Shape.drop([u'Address', u'BldgClass', u'BldgDepth', u'ComArea', u'FactryArea',
              u'FireComp', u'GarageArea', u'HealthArea', u'Lot',u'LotDepth',u'LotFront',
              u'LotType',u'OfficeArea',  u'OtherArea',  u'OwnerType',u'PolicePrct',   
              u'ProxCode',    u'ResArea', u'RetailArea',u'SHAPE_Leng', u'SchoolDist',  
              u'StrgeArea', u'UnitsRes',u'ZoneDist1'],axis = 1)
BK_Shape_2.columns

Index([  u'BldgArea',  u'BldgFront',      u'Block',     u'CT2010',
          u'LandUse',    u'LotArea', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'SHAPE_Area', u'UnitsTotal',     u'XCoord',
           u'YCoord',  u'YearBuilt',    u'ZipCode',   u'geometry'],
      dtype='object')

In [11]:
BK_Shape_2.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry
0,0,0.0,7036,336,11,81828,0,0,0.0,48254.897890,0,0,0,0,11224,"POLYGON ((-74.012372767644 40.57744691763495, ..."
1,0,0.0,7037,336,11,38010,0,0,0.0,12182.420391,0,0,0,0,11224,POLYGON ((-74.01237940630934 40.57657598861726...
2,0,0.0,8696,352,11,2350,0,0,0.0,2235.111932,0,990121,148530,0,11224,POLYGON ((-73.97880723690464 40.57467706266272...
3,3610,30.0,7029,336,02,4000,0,1,3.0,3979.291146,3,982905,148690,1920,11224,POLYGON ((-74.00473188788037 40.57469753965819...
4,1854,15.0,7049,342,01,1823,0,1,3.0,1683.769587,1,984915,148767,1999,11224,POLYGON ((-73.99754223950133 40.57492465276304...


In [17]:
BK_Shape_joined = gpd.sjoin(BK_Shape_2, qscore_geo, how="inner", op='intersects')
BK_Shape_joined.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry,index_right,latitude,longitude,q-score
5,225,0.0,7067,340,09,29930,0,1,1.0,63392.853235,0,0,0,1950,11224,POLYGON ((-73.99511484438293 40.57161058964105...,71472,40.571880,-73.996712,24.017101
5,225,0.0,7067,340,09,29930,0,1,1.0,63392.853235,0,0,0,1950,11224,POLYGON ((-73.99511484438293 40.57161058964105...,71475,40.571880,-73.995972,23.352837
13,380000,95.0,7055,326,03,153383,0,1,19.0,155218.397794,360,986630,148618,1972,11224,POLYGON ((-73.99201693083842 40.57543784797349...,71689,40.573624,-73.991646,32.690609
13,380000,95.0,7055,326,03,153383,0,1,19.0,155218.397794,360,986630,148618,1972,11224,POLYGON ((-73.99201693083842 40.57543784797349...,71731,40.573956,-73.991081,24.378029
13,380000,95.0,7055,326,03,153383,0,1,19.0,155218.397794,360,986630,148618,1972,11224,POLYGON ((-73.99201693083842 40.57543784797349...,71732,40.573956,-73.991455,34.001518


In [19]:
BK_Shape_joined_2 = gpd.sjoin(qscore_geo, BK_Shape_2, how="inner", op='intersects')
BK_Shape_joined_2.head()

,latitude,longitude,q-score,geometry,index_right,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode
71427,40.571148,-73.997086,10.854957,POINT (-73.997086 40.571148),118,3825,0.0,7065,352,09,7838727,0,8,1.0,3.002445e+06,0,987054,147054,1928,11224
71428,40.571148,-73.996719,13.955506,POINT (-73.996719 40.571148),118,3825,0.0,7065,352,09,7838727,0,8,1.0,3.002445e+06,0,987054,147054,1928,11224
71429,40.571159,-73.995979,11.594976,POINT (-73.99597900000001 40.571159),118,3825,0.0,7065,352,09,7838727,0,8,1.0,3.002445e+06,0,987054,147054,1928,11224
71430,40.571167,-73.995605,12.206305,POINT (-73.995605 40.571167),118,3825,0.0,7065,352,09,7838727,0,8,1.0,3.002445e+06,0,987054,147054,1928,11224
71431,40.571175,-73.998177,11.686462,POINT (-73.998177 40.571175),118,3825,0.0,7065,352,09,7838727,0,8,1.0,3.002445e+06,0,987054,147054,1928,11224


In [20]:
BK_Shape_joined.to_csv('BK_Geometry_Joined.csv')

In [21]:
BK_Shape_joined_2.to_csv('BK_Point_Joined.csv')